<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# Working with Watson Machine Learning

This notebook should be run in a Watson Studio project, using **Default Python 3.8** runtime environment. It requires service credentials for the following Cloud services:
  * Watson OpenScale 
  * Watson Machine Learning 
  * Cloud Object Storage
  
If you have a paid Cloud account, you may also provision a **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

### Contents

- [Setup](#setup)
- [Model building and deployment](#model)
- [OpenScale configuration](#openscale)
- [Quality monitor and feedback logging](#quality)
- [Fairness,Drift monitoring and explanations](#fairness)
- [Custom monitors and metrics](#custom)
- [Historical data](#historical)

# Setup <a name="setup"></a>

## Package installation
**NOTE** Using scikit-learn 0.20.2 is requirement for Drift detection model training. if you are training drift detection model using notebook, make sure you have scikit-learn version 0.20.2. Your main model can be of any scikit-learn framework version supported by Watson Machine Learning. 

In [ ]:
!pip install --upgrade ibm-watson-machine-learning | tail -n 1
!pip install --upgrade ibm-watson-openscale | tail -n 1

### Action: restart the kernel!

## Configure credentials

- WOS_CREDENTIALS (CP4D)
- WML_CREDENTIALS (CP4D)
- DB_CREDENTIALS (DB2 on CP4D)
- SCHEMA_NAME
- WML_SPACE_ID

In [1]:
WOS_CREDENTIALS = {
    "url": "",
    "username": "",
    "password": ""
}

### WML credentials example with API key

In [2]:
WML_CREDENTIALS = {
                   "url": WOS_CREDENTIALS["url"],
                   "username": WOS_CREDENTIALS["username"],
                   "password" : WOS_CREDENTIALS["password"],
                   "instance_id": "wml_local",
                   "version" : "4.0" #If your env is CP4D 4.0 then specify "4.0" instead of "3.5"
                  }

In [3]:
DB_CREDENTIALS = {
    "hostname":"***",
    "username":"***",
    "password":"***",
    "database":"***",
    "port":50000, #provide your actual DB2 port number (as integer value)
    "ssl":"***",
    "sslmode":"***",
    "certificate_base64":"***"}



### Action: Specify created schema name below.

In [4]:
SCHEMA_NAME = 'AIOSFASTPATHICP'

In next cells, you will need to paste some credentials to Cloud Object Storage. If you haven't worked with COS yet please visit getting started with COS tutorial. You can find `COS_API_KEY_ID` and `COS_RESOURCE_CRN` variables in **_Service Credentials_** in menu of your COS instance. Used COS Service Credentials must be created with _Role_ parameter set as Writer. Later training data file will be loaded to the bucket of your instance and used as training refecence in subsription.  
`COS_ENDPOINT` variable can be found in **_Endpoint_** field of the menu.

In [5]:
COS_API_KEY_ID = "***"
COS_RESOURCE_CRN = "***"
COS_ENDPOINT = "***" # Current list avaiable at https://control.cloud-object-storage.cloud.ibm.com/v2/endpoints
COS_IAM_AUTH_ENDPOINT = "https://iam.ng.bluemix.net/oidc/token" 

In [6]:
BUCKET_NAME = "" #example: "credit-risk-training-data" 

## Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Model building and deployment <a name="model"></a>

In this section you will learn how to train Scikit-learn model and next deploy it as web-service using Watson Machine Learning service.

## Load the training data from github

In [7]:
!rm german_credit_data_biased_training.csv
#!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv    

--2024-08-07 15:26:04--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/Cloud%20Pak%20for%20Data/WML/assets/data/credit_risk/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

german_credit_data_ 100%[===================>] 673.46K  --.-KB/s    in 0.09s   

2024-08-07 15:26:04 (7.10 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [8]:
import numpy as np
import pandas as pd

training_data_file_name = "german_credit_data_biased_training.csv"
data_df = pd.read_csv(training_data_file_name)

## Explore data

In [9]:
data_df.head()

,CheckingStatus,LoanDuration,CreditHistory,LoanPurpose,LoanAmount,ExistingSavings,EmploymentDuration,InstallmentPercent,Sex,OthersOnLoan,...,OwnsProperty,Age,InstallmentPlans,Housing,ExistingCreditsCount,Job,Dependents,Telephone,ForeignWorker,Risk
0,0_to_200,31,credits_paid_to_date,other,1889,100_to_500,less_1,3,female,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
1,less_0,18,credits_paid_to_date,car_new,462,less_100,1_to_4,2,female,none,...,savings_insurance,37,stores,own,2,skilled,1,none,yes,No Risk
2,less_0,15,prior_payments_delayed,furniture,250,less_100,1_to_4,2,male,none,...,real_estate,28,none,own,2,skilled,1,yes,no,No Risk
3,0_to_200,28,credits_paid_to_date,retraining,3693,less_100,greater_7,3,male,none,...,savings_insurance,32,none,own,1,skilled,1,none,yes,No Risk
4,no_checking,28,prior_payments_delayed,education,6235,500_to_1000,greater_7,3,male,none,...,unknown,57,none,own,2,skilled,1,none,yes,Risk


In [10]:
print('Columns: ', list(data_df.columns))
print('Number of columns: ', len(data_df.columns))

Columns:  ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'Risk']
Number of columns:  21


As you can see, the data contains twenty one fields. `Risk` field is the one you would like to predict using feedback data.

In [11]:
print('Number of records: ', data_df.Risk.count())

Number of records:  5000


In [12]:
target_count = data_df.groupby('Risk')['Risk'].count()
target_count

Risk
No Risk    3330
Risk       1670
Name: Risk, dtype: int64

## Save training data to Cloud Object Storage

In [13]:
import ibm_boto3
from ibm_botocore.client import Config, ClientError

cos_client = ibm_boto3.resource("s3",
    ibm_api_key_id=COS_API_KEY_ID,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    #ibm_auth_endpoint=COS_IAM_AUTH_ENDPOINT,
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

In [14]:
with open(training_data_file_name, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, training_data_file_name).upload_fileobj(
        Fileobj=file_data
    )

## Create a model
In this section you will learn how to:

- Prepare data for training a model
- Create machine learning pipeline
- Train a model

In [15]:
MODEL_NAME = "Scikit German Risk Model WML V4 - zLinux"
DEPLOYMENT_NAME = "Scikit German Risk Deployment WML V4 - zLinux"

### You will start with importing required libraries

In [16]:
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.decomposition import TruncatedSVD
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

### Splitting the data into train and test

In [17]:
train_data, test_data = train_test_split(data_df, test_size=0.2)

### Preparing the pipeline

In [18]:
features_idx = np.s_[0:-1]
all_records_idx = np.s_[:]
first_record_idx = np.s_[0]

In this step you will encode target column labels into numeric values. You can use `inverse_transform` to decode numeric predictions into labels.

In [19]:
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]
ct = ColumnTransformer([("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))])
clf_linear = SGDClassifier(loss='log', penalty='l2', max_iter=1000, tol=1e-5)

pipeline_linear = Pipeline([('ct', ct), ('clf_linear', clf_linear)])

### Train a model

In [20]:
risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)

/usr/local/lib/python3.10/site-packages/sklearn/linear_model/_stochastic_gradient.py:173: FutureWarning: The loss 'log' was deprecated in v1.1 and will be removed in version 1.3. Use `loss='log_loss'` which is equivalent.
  warnings.warn(


### Evaluate the model

In [21]:
from sklearn.metrics import roc_auc_score

predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction=='No Risk' else 1 for prediction in predictions]

real_observations = test_data.Risk.replace('Risk', 1)
real_observations = real_observations.replace('No Risk', 0).values

auc = roc_auc_score(real_observations, indexed_preds)
print(auc)

0.7416473359587132


## Publish the model

In this section, the notebook uses the supplied Watson Machine Learning credentials to save the model (including the pipeline) to the WML instance. Previous versions of the model are removed so that the notebook can be run again, resetting all data for another demo.

In [22]:
import json
from ibm_watson_machine_learning import APIClient

wml_client = APIClient(WML_CREDENTIALS)
wml_client.version

/usr/local/lib/python3.10/site-packages/ibm_watson_machine_learning/client.py:184: UserWarning: The provided version: 4.0 is different from the current CP4D version: 5.0. Correct the wml_credentials with proper CP4D version number.
  warn(


'1.0.360'

### Listing all the available spaces

In [23]:
wml_client.spaces.list(limit=10)

------------------------------------  -------------------------------------------------------------------  ------------------------
ID                                    NAME                                                                 CREATED
7e5a8be6-9103-4c22-9c43-b66f3d8364de  poojitha_notebooks_space                                             2024-06-29T13:53:46.645Z
16ccd855-46bd-43ed-8219-5f00ac565d08  shreya-space                                                         2024-06-26T04:29:17.302Z
bc3b9797-c509-4fb4-a424-f67b1e2ed4be  QUALITY_WMLV4_PREPROD                                                2024-06-23T12:23:04.790Z
e396e187-2977-47b4-ade3-1539f9f10adc  QUALITY_WMLV4_PROD                                                   2024-06-23T12:22:54.422Z
40c4d032-0339-4da6-bfec-4bdb096c9650  shreya                                                               2024-06-20T10:54:20.088Z
088c142e-f35e-4e48-a30c-ad55a6edeecc  notebooks 5.0                                          

,ID,NAME,CREATED
0,7e5a8be6-9103-4c22-9c43-b66f3d8364de,poojitha_notebooks_space,2024-06-29T13:53:46.645Z
1,16ccd855-46bd-43ed-8219-5f00ac565d08,shreya-space,2024-06-26T04:29:17.302Z
2,bc3b9797-c509-4fb4-a424-f67b1e2ed4be,QUALITY_WMLV4_PREPROD,2024-06-23T12:23:04.790Z
3,e396e187-2977-47b4-ade3-1539f9f10adc,QUALITY_WMLV4_PROD,2024-06-23T12:22:54.422Z
4,40c4d032-0339-4da6-bfec-4bdb096c9650,shreya,2024-06-20T10:54:20.088Z
5,088c142e-f35e-4e48-a30c-ad55a6edeecc,notebooks 5.0,2024-06-13T04:42:07.336Z
6,b9b3d3b4-6e26-4e16-807d-e8bf5e7d6984,MRM_WMLV4_PREPROD,2024-06-12T15:49:26.571Z
7,d22e2b6b-917c-4427-a40c-1a439352a742,MRM_WMLV4_PROD,2024-06-12T15:49:16.185Z
8,ce15e0f6-be30-4349-af47-35ae15983bf1,openscale-express-path-preprod-00000000-0000-0...,2024-06-04T05:18:51.988Z
9,6264dc0e-087a-4dea-bcbc-6bd872b510fb,openscale-express-path-00000000-0000-0000-0000...,2024-06-04T05:18:30.811Z


In [24]:
WML_SPACE_ID='***' # use space id here
wml_client.set.default_space(WML_SPACE_ID)

'SUCCESS'

### Remove existing model and deployment

In [25]:
import time
deployments_list = wml_client.deployments.get_details()
for deployment in deployments_list["resources"]:
    model_id = deployment["entity"]["asset"]["id"]
    deployment_id = deployment["metadata"]["id"]
    if deployment["metadata"]["name"] == DEPLOYMENT_NAME:
        print("Deleting deployment id", deployment_id)
        wml_client.deployments.delete(deployment_id)
        time.sleep(5)
        print("Deleting model id", model_id)
        wml_client.repository.delete(model_id)
        time.sleep(5)

wml_client.repository.list_models()


Deleting deployment id 901d7578-58e6-44ac-aa11-7de2fdc1efa5
Deleting model id d45280e1-170c-4523-8b82-93a21eae4830
--  ----  -------  ----  ----------  ----------------
ID  NAME  CREATED  TYPE  SPEC_STATE  SPEC_REPLACEMENT
--  ----  -------  ----  ----------  ----------------


,ID,NAME,CREATED,TYPE,SPEC_STATE,SPEC_REPLACEMENT


In [26]:
training_data_references = [
                {
                    "id": "product line",
                    "type": "s3",
                    "connection": {
                        "access_key_id": COS_API_KEY_ID,
                        "endpoint_url": COS_ENDPOINT,
                        "resource_instance_id":COS_RESOURCE_CRN
                    },
                    "location": {
                        "bucket": BUCKET_NAME,
                        "path": training_data_file_name,
                    }
                }
            ]

In [27]:

software_spec_uid = wml_client.software_specifications.get_id_by_name("runtime-23.1-py3.10")
print("Software Specification ID: {}".format(software_spec_uid))

model_props = {
        wml_client._models.ConfigurationMetaNames.NAME:"{}".format(MODEL_NAME),
        wml_client._models.ConfigurationMetaNames.TYPE: "scikit-learn_1.1",
        wml_client._models.ConfigurationMetaNames.SOFTWARE_SPEC_UID: software_spec_uid,
        wml_client._models.ConfigurationMetaNames.LABEL_FIELD: "Risk",
    }

Software Specification ID: 336b29df-e0e1-5e7d-b6a5-f6ab722625b2


In [28]:
print("Storing model ...")

published_model_details = wml_client.repository.store_model(model=risk_model, meta_props=model_props, training_data=data_df.drop(["Risk"], axis=1), training_target=data_df.Risk)
model_uid = wml_client.repository.get_model_id(published_model_details)
print("Done")
print("Model ID: {}".format(model_uid))

Storing model ...
Done
Model ID: 1b9f22c5-ac03-4017-b40b-27969e429379


In [29]:
model_details = wml_client.repository.get_details(model_uid)
print(json.dumps(model_details, indent=2))

{
  "entity": {
    "hybrid_pipeline_software_specs": [],
    "label_column": "Risk",
    "schemas": {
      "input": [
        {
          "fields": [
            {
              "name": "CheckingStatus",
              "type": "object"
            },
            {
              "name": "LoanDuration",
              "type": "int64"
            },
            {
              "name": "CreditHistory",
              "type": "object"
            },
            {
              "name": "LoanPurpose",
              "type": "object"
            },
            {
              "name": "LoanAmount",
              "type": "int64"
            },
            {
              "name": "ExistingSavings",
              "type": "object"
            },
            {
              "name": "EmploymentDuration",
              "type": "object"
            },
            {
              "name": "InstallmentPercent",
              "type": "int64"
            },
            {
              "name": "Sex",
         

## Deploy the model

The next section of the notebook deploys the model as a RESTful web service in Watson Machine Learning. The deployed model will have a scoring URL you can use to send data to the model for predictions.

In [30]:
print("Deploying model...")
deployment_details = wml_client.deployments.create(
    model_uid, 
    meta_props={
        wml_client.deployments.ConfigurationMetaNames.NAME: "{}".format(DEPLOYMENT_NAME),
        wml_client.deployments.ConfigurationMetaNames.ONLINE: {}
    }
)
scoring_url = wml_client.deployments.get_scoring_href(deployment_details)
deployment_uid=wml_client.deployments.get_uid(deployment_details)

print("Scoring URL:" + scoring_url)
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '1b9f22c5-ac03-4017-b40b-27969e429379' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='edc1e06c-7bf9-4522-8893-05a919a35ca0'
------------------------------------------------------------------------------------------------


Scoring URL:https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/ml/v4/deployments/edc1e06c-7bf9-4522-8893-05a919a35ca0/predictions
Model id: 1b9f22c5-ac03-4017-b40b-27969e429379
Deployment id: edc1e06c-7bf9-4522-8893-05a919a35ca0


## Score the model

In [31]:
fields = ["CheckingStatus", "LoanDuration", "CreditHistory", "LoanPurpose", "LoanAmount", "ExistingSavings",
                  "EmploymentDuration", "InstallmentPercent", "Sex", "OthersOnLoan", "CurrentResidenceDuration",
                  "OwnsProperty", "Age", "InstallmentPlans", "Housing", "ExistingCreditsCount", "Job", "Dependents",
                  "Telephone", "ForeignWorker"]
values = [
            ["no_checking", 13, "credits_paid_to_date", "car_new", 1343, "100_to_500", "1_to_4", 2, "female", "none", 3,
             "savings_insurance", 46, "none", "own", 2, "skilled", 1, "none", "yes"],
            ["no_checking", 24, "prior_payments_delayed", "furniture", 4567, "500_to_1000", "1_to_4", 4, "male", "none",
             4, "savings_insurance", 36, "none", "free", 2, "management_self-employed", 1, "none", "yes"],
        ]

scoring_payload = {"input_data": [{"fields": fields, "values": values}]}

In [32]:
predictions = wml_client.deployments.score(deployment_uid, scoring_payload)
predictions

{'predictions': [{'fields': ['prediction', 'probability'],
   'values': [['No Risk', [0.565773002160447, 0.43422699783955293]],
    ['No Risk', [0.8392812451374476, 0.16071875486255235]]]}]}

# Configure OpenScale <a name="openscale"></a>

The notebook will now import the necessary libraries and set up a Python OpenScale client.

In [33]:
from ibm_cloud_sdk_core.authenticators import CloudPakForDataAuthenticator

from ibm_watson_openscale import *
from ibm_watson_openscale.supporting_classes.enums import *
from ibm_watson_openscale.supporting_classes import *


authenticator = CloudPakForDataAuthenticator(
        url=WOS_CREDENTIALS['url'],
        username=WOS_CREDENTIALS['username'],
        password=WOS_CREDENTIALS['password'],
        disable_ssl_verification=True
    )
#Create client for the default instance id 00000000-0000-0000-0000-000000000000
wos_client = APIClient(service_url=WOS_CREDENTIALS['url'], authenticator=authenticator)
wos_client.version

'3.0.40'

## Create schema and datamart

### Set up datamart

Watson OpenScale uses a database to store payload logs and calculated metrics. If database credentials were supplied, the datamart will be created there unless there is an existing datamart. If an OpenScale datamart exists, the existing datamart will be used and no data will be overwritten.

Prior instances of the German Credit model will be removed from OpenScale monitoring.

In [34]:
wos_client.data_marts.show()

AIOSFASTPATHICP-00000000-0000-0000-0000-000000000000,Data Mart created by OpenScale ExpressPath,False,active,2024-06-04 05:19:03.698000+00:00,00000000-0000-0000-0000-000000000000


In [35]:
data_marts = wos_client.data_marts.list().result.data_marts
if len(data_marts) == 0:
    if DB_CREDENTIALS is not None:
        if SCHEMA_NAME is None: 
            print("Please specify the SCHEMA_NAME and rerun the cell")

        print('Setting up external datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook",
                database_configuration=DatabaseConfigurationRequest(
                  database_type=DatabaseType.POSTGRESQL,
                    credentials=PrimaryStorageCredentialsLong(
                        hostname=DB_CREDENTIALS['hostname'],
                        username=DB_CREDENTIALS['username'],
                        password=DB_CREDENTIALS['password'],
                        db=DB_CREDENTIALS['database'],
                        port=DB_CREDENTIALS['port'],
                        ssl=True,
                        sslmode=DB_CREDENTIALS['sslmode'],
                        certificate_base64=DB_CREDENTIALS['certificate_base64']
                    ),
                    location=LocationSchemaName(
                        schema_name= SCHEMA_NAME
                    )
                )
             ).result
    else:
        print('Setting up internal datamart')
        added_data_mart_result = wos_client.data_marts.add(
                background_mode=False,
                name="WOS Data Mart",
                description="Data Mart created by WOS tutorial notebook", 
                internal_database = True).result
        
    data_mart_id = added_data_mart_result.metadata.id
    
else:
    data_mart_id=data_marts[0].metadata.id
    print('Using existing datamart {}'.format(data_mart_id))

Using existing datamart 00000000-0000-0000-0000-000000000000


### Remove existing service provider connected with used  WML instance. 

Multiple service providers for the same engine instance are avaiable in Watson OpenScale. To avoid multiple service providers of used WML instance in the tutorial notebook the following code deletes existing service provder(s) and then adds new one. 

In [36]:
SERVICE_PROVIDER_NAME = "Watson Machine Learning V2"
SERVICE_PROVIDER_DESCRIPTION = "Added by tutorial WOS notebook."

In [37]:
service_providers = wos_client.service_providers.list().result.service_providers
for service_provider in service_providers:
    service_instance_name = service_provider.entity.name
    if service_instance_name == SERVICE_PROVIDER_NAME:
        service_provider_id = service_provider.metadata.id
        wos_client.service_providers.delete(service_provider_id)
        print("Deleted existing service_provider for WML instance: {}".format(service_provider_id))

## Add service provider

Watson OpenScale needs to be bound to the Watson Machine Learning instance to capture payload data into and out of the model.

**Note:** You can bind more than one engine instance if needed by calling `wos_client.service_providers.add` method. Next, you can refer to particular service provider using `service_provider_id`.

In [38]:
added_service_provider_result = wos_client.service_providers.add(
        name=SERVICE_PROVIDER_NAME,
        description=SERVICE_PROVIDER_DESCRIPTION,
        service_type=ServiceTypes.WATSON_MACHINE_LEARNING,
        deployment_space_id = WML_SPACE_ID,
        operational_space_id = "production",
        credentials=WMLCredentialsCP4D(),
        background_mode=False
    ).result
service_provider_id = added_service_provider_result.metadata.id




 Waiting for end of adding service provider cf31400f-925c-4180-b4b2-77e53440d9a7 




active

-----------------------------------------------
 Successfully finished adding service provider 
-----------------------------------------------




In [39]:
wos_client.service_providers.show()

99999999-9999-9999-9999-999999999999,active,Watson Machine Learning V2-2,watson_machine_learning,2024-08-07 09:57:15.265000+00:00,cf31400f-925c-4180-b4b2-77e53440d9a7
99999999-9999-9999-9999-999999999999,active,WML - for training data -2,watson_machine_learning,2024-08-07 08:42:56.271000+00:00,1b7f6643-f6ac-4754-9f39-e8dbe5b09232
99999999-9999-9999-9999-999999999999,active,Watson Machine Learning V2_test-2,watson_machine_learning,2024-08-07 06:51:26.963000+00:00,b9fd3cb5-c7a4-4b5c-a90c-684676276352
99999999-9999-9999-9999-999999999999,active,WML Prod,watson_machine_learning,2024-08-07 06:34:58.909000+00:00,f4931b54-2103-40d9-b564-9ba3b1982fdb
99999999-9999-9999-9999-999999999999,active,WML Pre-Prod2,watson_machine_learning,2024-08-07 06:03:11.179000+00:00,81655d73-6ea3-44ae-9876-86df1fa6390d
99999999-9999-9999-9999-999999999999,active,WML AI function - WOS notebook,watson_machine_learning,2024-08-06 13:32:38.884000+00:00,e413a76d-802c-44f3-8c55-300c5aa2659b
99999999-9999-9999-9999-999999999999,active,Watson Machine Learning - Indirect Bias Demo-2,watson_machine_learning,2024-08-06 13:25:37.046000+00:00,c3601bc6-22c9-45a1-b016-c29cf0bf64b4
None,active,IAE7,custom_machine_learning,2024-07-02 09:08:06.273000+00:00,184e73a2-7fd8-4f3f-b994-bb648f6eb8ec
None,active,IAE6,custom_machine_learning,2024-07-02 09:04:22.643000+00:00,644befcd-6d36-4f4d-a30a-cd51a28b63fe
None,active,WML_IAE5,custom_machine_learning,2024-07-02 08:43:08.723000+00:00,e986a0d7-8187-4fed-ab2a-c614a9683cae


Note: First 10 records were displayed.


## Subscriptions

### Remove existing credit risk subscriptions

In [ ]:
wos_client.subscriptions.show()

This code removes previous subscriptions to the German Credit model to refresh the monitors with the new model and new data.

In [40]:
subscriptions = wos_client.subscriptions.list().result.subscriptions
for subscription in subscriptions:
    sub_model_id = subscription.entity.asset.asset_id
    if sub_model_id == model_uid:
        wos_client.subscriptions.delete(subscription.metadata.id)
        print("Deleted existing subscription for model", sub_model_id)

This code creates the model subscription in OpenScale using the Python client API. Note that we need to provide the model unique identifier, and some information about the model itself.

In [41]:
ASSET_NAME = "Scikit German Risk Model WML V4 - zLinux"
asset = Asset(
    asset_id=model_uid,
    name = ASSET_NAME,
    url=deployment_details["entity"]["status"]["online_url"]["url"],
    asset_type=AssetTypes.MODEL,
    input_data_type=InputDataType.STRUCTURED,
    problem_type=ProblemType.BINARY_CLASSIFICATION
)
asset_deployment = AssetDeploymentRequest(
    deployment_id=deployment_uid,
    name=DEPLOYMENT_NAME,
    deployment_type=DeploymentTypes.ONLINE,
    url=deployment_details["entity"]["status"]["online_url"]["url"]
)
training_data_reference = TrainingDataReference(
    type="cos",
    location=COSTrainingDataReferenceLocation(
        bucket=BUCKET_NAME,
        file_name=training_data_file_name
    ),
    connection=COSTrainingDataReferenceConnection.from_dict(
        {
            "resource_instance_id": COS_RESOURCE_CRN,
            "url": COS_ENDPOINT,
            "api_key": COS_API_KEY_ID,
            "iam_url": COS_IAM_AUTH_ENDPOINT
        }
    )
)
training_fields = []
for field in published_model_details["entity"]["training_data_references"][0]["schema"]["fields"]:
    field_name = field["name"]
    field_type = field["type"]
    if field_type == "object":
        field_type = "string"
    elif field_type == "int64":
        field_type = "integer"
    elif field_type == "float64":
        field_type = "double"
    field_object = SparkStructFieldObject(
        name=field_name,
        type=field_type,
        nullable=False,
        metadata=None
    )
    training_fields.append(field_object)
training_data_schema = SparkStruct(
    type="struct",
    fields=training_fields
)
asset_properties_request = AssetPropertiesRequest(
    label_column="Risk",
    probability_fields=["probability"],
    prediction_field="prediction",
    feature_fields=["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_fields=["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"],
    training_data_reference=training_data_reference
    #,training_data_schema=training_data_schema
)


In [42]:
subscription_details = wos_client.subscriptions.add(
        data_mart_id=data_mart_id,
        service_provider_id=service_provider_id,
        asset=asset,
        deployment=asset_deployment,
        asset_properties=asset_properties_request,background_mode = False).result
subscription_id = subscription_details.metadata.id
print(subscription_details)




 Waiting for end of adding subscription ad19393e-2078-4316-ac19-7de0ddfd7937 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


{
  "metadata": {
    "id": "ad19393e-2078-4316-ac19-7de0ddfd7937",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:subscription:ad19393e-2078-4316-ac19-7de0ddfd7937",
    "url": "/v2/subscriptions/ad19393e-2078-4316-ac19-7de0ddfd7937",
    "created_at": "2024-08-07T09:57:28.433000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-08-07T09:57:30.212000Z",
    "modified_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "service_provider_id": "cf31400f-925c-4180-b4b2-77e53440d9a7",
    "asset": {
      "asset_id": "1b9f22c5-ac03-4017-b40b-27969e429379",
      "url": "https://cpd-cpd-instance.apps.wos415nfs2672.cp.fyre.ibm.com/ml/v4/deployments/edc1e06c-7bf9-4522-

In [43]:
import time

time.sleep(5)
payload_data_set_id = None
payload_data_set_id = wos_client.data_sets.list(type=DataSetTypes.PAYLOAD_LOGGING, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result.data_sets[0].metadata.id
if payload_data_set_id is None:
    print("Payload data set not found. Please check subscription status.")
else:
    print("Payload data set id:", payload_data_set_id)

Payload data set id: 6405b132-9c0e-4685-bf12-2d97d80f76f2


In [44]:
wos_client.data_sets.show()

00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,model_health,2024-08-07 09:57:30.279000+00:00,e157912c-dca7-4e7e-9e21-855abb979ef6
00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,payload_logging_error,2024-08-07 09:57:30.116000+00:00,e6ae4454-bdcd-47da-b821-49f3be904ea0
00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,manual_labeling,2024-08-07 09:57:29.870000+00:00,6e9fe31e-3316-40a4-b011-da5dacd7ec84
00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,payload_logging,2024-08-07 09:57:29.681000+00:00,6405b132-9c0e-4685-bf12-2d97d80f76f2
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,manual_labeling,2024-08-07 08:51:04.012000+00:00,87a1b239-525c-4170-b35c-ea039e8b8778
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,explanations,2024-08-07 08:52:08.778000+00:00,0af7a298-cb89-4a6d-af33-2a1ab41146f1
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,model_health,2024-08-07 08:51:04.164000+00:00,55c3cd26-599a-4a0c-b14b-d3afbf488ac9
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,payload_logging,2024-08-07 08:51:03.814000+00:00,16494c6f-c675-4da5-824f-330a3ab23cb8
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,payload_logging_error,2024-08-07 08:51:04.621000+00:00,0d6e0513-57bb-44bb-8e13-482c974d457a
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,training,2024-08-07 08:51:04.203000+00:00,ad83e976-366f-4758-861a-e733f44db73f


Note: First 10 records were displayed.




Get subscription list

In [45]:
wos_client.subscriptions.show()

1b9f22c5-ac03-4017-b40b-27969e429379,model,Scikit German Risk Model WML V4 - zLinux,00000000-0000-0000-0000-000000000000,edc1e06c-7bf9-4522-8893-05a919a35ca0,Scikit German Risk Deployment WML V4 - zLinux,cf31400f-925c-4180-b4b2-77e53440d9a7,active,2024-08-07 09:57:28.433000+00:00,ad19393e-2078-4316-ac19-7de0ddfd7937
df904b8a-2884-4318-82df-35cb430f8587,model,Scikit German Risk Model - for training data - zLinux,00000000-0000-0000-0000-000000000000,17e4097d-c4b5-41fe-bba5-3e12af95041e,Scikit German Risk Model - for training data - zLinux,1b7f6643-f6ac-4754-9f39-e8dbe5b09232,active,2024-08-07 08:51:02.441000+00:00,bcbbcc1f-7e1c-4746-8055-128ce8d805ff
20516963-37ff-4769-a41f-b509827f3076,model,Text Binary Classifier,00000000-0000-0000-0000-000000000000,18c8f8d4-cfbf-4fbb-a51a-5ac59d2e9e20,Text Binary Classifier deployment,b9fd3cb5-c7a4-4b5c-a90c-684676276352,active,2024-08-07 06:51:41.900000+00:00,d673e127-dd69-4b05-a204-5b90332d7e72
67e11064-81b2-4ea8-addf-b39c218299b4,model,German Credit Risk Model - Challenger,00000000-0000-0000-0000-000000000000,978e26e1-509c-4e03-b20a-baacfbbe222b,German Credit Risk Model - Challenger,81655d73-6ea3-44ae-9876-86df1fa6390d,active,2024-08-07 06:07:38.473000+00:00,c0174a6c-d8e6-458b-8549-cd03d7df486e
f1b4fe35-1a75-4dca-a9b3-bc5dd7ed43db,model,German Credit Risk Model - PreProd,00000000-0000-0000-0000-000000000000,42b13344-baa1-45a0-89ca-ef662387f691,German Credit Risk Model - PreProd,81655d73-6ea3-44ae-9876-86df1fa6390d,active,2024-08-07 06:07:24.153000+00:00,61003e50-20d6-4ce8-a379-013a03c44229
a4a1a71d-2a44-43f8-b92a-ba3c80dc36fd,model,Credit Risk python Fn Model,00000000-0000-0000-0000-000000000000,90b19695-de03-41b9-b820-2d3e11c2f662,dep_Credit Risk python Fn Model,e413a76d-802c-44f3-8c55-300c5aa2659b,active,2024-08-06 13:32:53.985000+00:00,77c99979-8408-40b7-ab3a-e3c0a8a80aab
eed8b225-19e5-4460-9b6f-7271dc1e3ff2,model,Adult Census Income Classifier Model,00000000-0000-0000-0000-000000000000,038b8c68-6615-4c0d-8d46-15783d6ab9c8,Adult Census Income Classifier Deployment,c3601bc6-22c9-45a1-b016-c29cf0bf64b4,active,2024-08-06 13:26:09.549000+00:00,c274e76c-6bb9-43ce-ba64-127a3db95ede
438ca544-9bd1-48c2-8e8d-3de4ef4ca79b,model,WML_IAE4,00000000-0000-0000-0000-000000000000,78a0af9e-1014-4fb1-b22a-5e11f4fd70e7,WML_IAE4,d90c6bf2-49c6-4179-9876-8b85b0247d95,active,2024-07-02 07:03:31.504000+00:00,e34b9b87-b6e1-4c53-b92e-cb80dea042be
592b902d-3dc9-4e56-8bcb-86cbf1a6d8a9,model,gcr - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,2b976af0-e4ab-4859-af7d-2f2287d864ad,gcr model,4d2f2fb2-6b64-4d58-8f13-257166e468e9,active,2024-07-17 07:11:44.727000+00:00,e2df4ec7-6c75-416f-a444-8d21389f7513
e3ac9fc3-bccf-4a4e-b37b-490bfb93dd81,model,GCR AutoAI - P2 XGB Classifier - Model,00000000-0000-0000-0000-000000000000,6399e6e8-df5a-4370-9af4-34b2f2e76bc6,GCR Auto AI,a7ca157a-de07-457a-8c4c-b1a2e998699c,active,2024-07-03 10:37:20.487000+00:00,ce36911c-75f6-4f99-b4d2-b71e5d55a802


Note: First 10 records were displayed.


### Score the model so we can configure monitors

Now that the WML service has been bound and the subscription has been created, we need to send a request to the model before we configure OpenScale. This allows OpenScale to create a payload log in the datamart with the correct schema, so it can capture data coming into and out of the model. The sends a few records for predictions.

In [46]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",10,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",20,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",24,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",17,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",66,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",14,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",35,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",23,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"input_data": [{"fields": fields, "values": values}]}
predictions = wml_client.deployments.score(deployment_uid, payload_scoring)

print("Single record scoring result:", "\n fields:", predictions["predictions"][0]["fields"], "\n values: ", predictions["predictions"][0]["values"][0])

Single record scoring result: 
 fields: ['prediction', 'probability'] 
 values:  ['No Risk', [0.565773002160447, 0.43422699783955293]]


## Check if WML payload logging worked else manually store payload records

In [47]:
import uuid
from ibm_watson_openscale.supporting_classes.payload_record import PayloadRecord
time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
print("Number of records in the payload logging table: {}".format(pl_records_count))
if pl_records_count == 0:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": predictions['predictions'][0]['fields'], "values":predictions['predictions'][0]['values']},
                   response_time=460
               )],background_mode = False)
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

Number of records in the payload logging table: 8


# Quality monitoring and feedback logging <a name="quality"></a>

## Enable quality monitoring

The code below waits ten seconds to allow the payload logging table to be set up before it begins enabling monitors. First, it turns on the quality (accuracy) monitor and sets an alert threshold of 70%. OpenScale will show an alert on the dashboard if the model accuracy measurement (area under the curve, in the case of a binary classifier) falls below this threshold.

The second paramater supplied, min_records, specifies the minimum number of feedback records OpenScale needs before it calculates a new measurement. The quality monitor runs hourly, but the accuracy reading in the dashboard will not change until an additional 50 feedback records have been added, via the user interface, the Python client, or the supplied feedback endpoint.

In [48]:
import time

time.sleep(10)
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
)
parameters = {
    "min_feedback_data_size": 50
}
thresholds = [
                {
                    "metric_id": "area_under_roc",
                    "type": "lower_limit",
                    "value": .80
                }
            ]
quality_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.QUALITY.ID,
    target=target,
    parameters=parameters,
    thresholds=thresholds
).result




 Waiting for end of monitor instance creation 30065556-8bab-4edb-85e4-f5b260bf0a3d 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [49]:
quality_monitor_instance_id = quality_monitor_details.metadata.id
quality_monitor_instance_id

'30065556-8bab-4edb-85e4-f5b260bf0a3d'

## Feedback logging

The code below downloads and stores enough feedback data to meet the minimum threshold so that OpenScale can calculate a new accuracy measurement. It then kicks off the accuracy monitor. The monitors run hourly, or can be initiated via the Python API, the REST API, or the graphical user interface.

In [50]:
!rm additional_feedback_data_v2.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json

rm: additional_feedback_data_v2.json: No such file or directory
--2024-08-07 15:28:16--  https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/additional_feedback_data_v2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 50890 (50K) [text/plain]
Saving to: ‘additional_feedback_data_v2.json’

additional_feedback 100%[===================>]  49.70K  --.-KB/s    in 0.02s   

2024-08-07 15:28:16 (3.17 MB/s) - ‘additional_feedback_data_v2.json’ saved [50890/50890]



### Get feedback logging dataset ID

In [51]:
feedback_dataset_id = None
feedback_dataset = wos_client.data_sets.list(type=DataSetTypes.FEEDBACK, 
                                                target_target_id=subscription_id, 
                                                target_target_type=TargetTypes.SUBSCRIPTION).result
print(feedback_dataset)
feedback_dataset_id = feedback_dataset.data_sets[0].metadata.id
if feedback_dataset_id is None:
    print("Feedback data set not found. Please check quality monitor status.")

{
  "data_sets": [
    {
      "metadata": {
        "id": "968d1f13-d172-44bb-9c45-636fb35a392c",
        "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:data_set:968d1f13-d172-44bb-9c45-636fb35a392c",
        "url": "/v2/data_sets/968d1f13-d172-44bb-9c45-636fb35a392c",
        "created_at": "2024-08-07T09:58:04.309000Z",
        "created_by": "internal-service",
        "modified_at": "2024-08-07T09:58:04.784000Z",
        "modified_by": "internal-service"
      },
      "entity": {
        "data_mart_id": "00000000-0000-0000-0000-000000000000",
        "name": "ad19393e-2078-4316-ac19-7de0ddfd7937_feedback",
        "description": "ad19393e-2078-4316-ac19-7de0ddfd7937_feedback",
        "type": "feedback",
        "target": {
          "target_type": "subscription",
          "target_id": "ad19393e-2078-4316-ac19-7de0ddfd7937"
        },
        "schema_update_mode": "auto",
        "data_schema": {
          "type": "struct",
          "

In [52]:
with open('additional_feedback_data_v2.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)

In [53]:
time.sleep(5)
wos_client.data_sets.store_records(feedback_dataset_id, request_body=additional_feedback_data, background_mode=False)




 Waiting for end of storing records with request id: f02d74bf-f00b-406d-89be-f6028bee7da8 




active

---------------------------------------
 Successfully finished storing records 
---------------------------------------




In [54]:
wos_client.data_sets.get_records_count(data_set_id=feedback_dataset_id)

98

In [55]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=quality_monitor_instance_id, background_mode=False).result




 Waiting for end of monitoring run 035f7c44-852c-4db2-a94a-d457d6e588ff 




finished

---------------------------
 Successfully finished run 
---------------------------




In [56]:
time.sleep(5)
wos_client.monitor_instances.show_metrics(monitor_instance_id=quality_monitor_instance_id)

2024-08-07 09:58:29.587000+00:00,true_positive_rate,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.42424242424242425,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,area_under_roc,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.6813519813519814,0.8,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,precision,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.7777777777777778,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,matthews_correlation_coefficient,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.4426747010626324,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,f1_measure,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.5490196078431373,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,accuracy,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.7653061224489796,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,label_skew,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.690933627340049,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,gini_coefficient,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.36270396270396277,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,log_loss,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.4547315703073755,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:58:29.587000+00:00,false_positive_rate,7e4eaaea-4e49-462b-b0eb-73885f6bc544,0.06153846153846154,None,None,['model_type:original'],quality,30065556-8bab-4edb-85e4-f5b260bf0a3d,035f7c44-852c-4db2-a94a-d457d6e588ff,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


Note: First 10 records were displayed.


# Fairness monitoring and explanations <a name="fairness"></a>

### Fairness configuration

The code below configures fairness monitoring for our model. It turns on monitoring for two features, Sex and Age. In each case, we must specify:
  * Which model feature to monitor
  * One or more **majority** groups, which are values of that feature that we expect to receive a higher percentage of favorable outcomes
  * One or more **minority** groups, which are values of that feature that we expect to receive a higher percentage of unfavorable outcomes
  * The threshold at which we would like OpenScale to display an alert if the fairness measurement falls below (in this case, 95%)

Additionally, we must specify which outcomes from the model are favourable outcomes, and which are unfavourable. We must also provide the number of records OpenScale will use to calculate the fairness score. In this case, OpenScale's fairness monitor will run hourly, but will not calculate a new fairness rating until at least 200 records have been added. Finally, to calculate fairness, OpenScale must perform some calculations on the training data, so we provide the dataframe containing the data.

In [57]:
wos_client.monitor_instances.show()

00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,quality,2024-08-07 09:58:03.805000+00:00,30065556-8bab-4edb-85e4-f5b260bf0a3d
00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,model_health,2024-08-07 09:57:29.730000+00:00,9b5fdace-ff32-4db1-a3eb-8eed7dfbea39
00000000-0000-0000-0000-000000000000,active,ad19393e-2078-4316-ac19-7de0ddfd7937,subscription,performance,2024-08-07 09:57:30.749000+00:00,9c497209-cf32-4cf0-b74d-ca560604f28b
00000000-0000-0000-0000-000000000000,active,a116845d-0dcc-4371-bf15-a793f6e3eedf,subscription,model_health,2024-06-20 13:56:46.752000+00:00,a726849f-eaa6-44c9-bfd3-a6e4215ea1b9
00000000-0000-0000-0000-000000000000,active,d330f8e4-8c21-4613-b957-0fbaea510978,subscription,model_health,2024-06-20 13:53:56.882000+00:00,635b7d77-ae32-4830-9be3-ac08125c1745
00000000-0000-0000-0000-000000000000,active,d673e127-dd69-4b05-a204-5b90332d7e72,subscription,model_health,2024-08-07 06:51:43.124000+00:00,f09d79c5-0ff6-4a79-8dd7-b63b729556f1
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,model_health,2024-08-07 08:51:03.669000+00:00,6992a022-94b0-49cb-8e46-1a7064836aa9
00000000-0000-0000-0000-000000000000,active,bcbbcc1f-7e1c-4746-8055-128ce8d805ff,subscription,fairness,2024-08-07 08:51:59.579000+00:00,a4d97b96-32a0-43eb-96f2-df364c2293b8
00000000-0000-0000-0000-000000000000,active,9db7eff3-d92f-462c-85c0-dcfb4f7725a0,subscription,model_health,2024-06-26 07:45:03.578000+00:00,c186c35b-e5ea-499d-a346-d9c3313bb41f
00000000-0000-0000-0000-000000000000,active,f4178567-4f9a-4b0e-b255-2babdb8b5f38,subscription,model_health,2024-07-02 08:43:26.098000+00:00,683a06d3-7cfb-4bd5-9f0d-ff67af9f5cf7


Note: First 10 records were displayed.


In [58]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)
parameters = {
    "features": [
        {"feature": "Sex",
         "majority": ['male'],
         "minority": ['female'],
         "threshold": 0.95
         },
        {"feature": "Age",
         "majority": [[26, 75]],
         "minority": [[18, 25]],
         "threshold": 0.95
         }
    ],
    "favourable_class": ["No Risk"],
    "unfavourable_class": ["Risk"],
    "min_records": 100
}

fairness_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.FAIRNESS.ID,
    target=target,
    parameters=parameters).result
fairness_monitor_instance_id =fairness_monitor_details.metadata.id




 Waiting for end of monitor instance creation da94306a-8542-4b49-b04d-f27b7ae77ec1 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Drift configuration

####  Note: you can choose to enable/disable (True or False) model or data drift within config

In [59]:
monitor_instances = wos_client.monitor_instances.list().result.monitor_instances
for monitor_instance in monitor_instances:
    monitor_def_id=monitor_instance.entity.monitor_definition_id
    if monitor_def_id == "drift" and monitor_instance.entity.target.target_id == subscription_id:
        wos_client.monitor_instances.delete(monitor_instance.metadata.id)
        print('Deleted existing drift monitor instance with id: ', monitor_instance.metadata.id)

In [60]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id

)

parameters = {
    "min_samples": 100,
    "drift_threshold": 0.1,
    "train_drift_model": True,
    "enable_model_drift": False,
    "enable_data_drift": True
}


drift_monitor_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.DRIFT.ID,
    target=target,
    parameters=parameters
).result

drift_monitor_instance_id = drift_monitor_details.metadata.id




 Waiting for end of monitor instance creation 505f727e-fa36-473e-91a5-e7f21cbabfc7 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Score the model again now that monitoring is configured

This next section randomly selects 200 records from the data feed and sends those records to the model for predictions. This is enough to exceed the minimum threshold for records set in the previous section, which allows OpenScale to begin calculating fairness.

In [61]:
from IPython.utils import io
!rm german_credit_feed.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_feed.json
!ls -lh german_credit_feed.json

-rw-r--r--  1 nelwin  staff   2.9M Aug  7 15:29 german_credit_feed.json


Score 200 randomly chosen records

In [62]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(200):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"input_data": [{"fields": fields, "values": values}]}

scoring_response = wml_client.deployments.score(deployment_uid, payload_scoring)

time.sleep(5)
pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)

if pl_records_count == 8:
    print("Payload logging did not happen, performing explicit payload logging.")
    wos_client.data_sets.store_records(data_set_id=payload_data_set_id, request_body=[PayloadRecord(
                   scoring_id=str(uuid.uuid4()),
                   request=payload_scoring,
                   response={"fields": scoring_response['predictions'][0]['fields'], "values":scoring_response['predictions'][0]['values']},
                   response_time=460
               )])
    time.sleep(5)
    pl_records_count = wos_client.data_sets.get_records_count(payload_data_set_id)
    print("Number of records in the payload logging table: {}".format(pl_records_count))

**Note:** Now in payload table should be total 208 records.

In [63]:
print("Number of records in payload table:", wos_client.data_sets.get_records_count(data_set_id=payload_data_set_id))

Number of records in payload table: 208


## Run fairness monitor

Kick off a fairness monitor run on current data. The monitor runs hourly, but can be manually initiated using the Python client, the REST API, or the graphical user interface.

In [64]:
run_details = wos_client.monitor_instances.run(monitor_instance_id=fairness_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run aff300d0-f16a-45cc-8201-a00790c1df94 




running.
finished

---------------------------
 Successfully finished run 
---------------------------




In [65]:
time.sleep(10)

wos_client.monitor_instances.show_metrics(monitor_instance_id=fairness_monitor_instance_id)

2024-08-07 09:59:24.165078+00:00,fairness_value,c9333fb3-82cd-431d-9eb5-65bd64c9f7bd,82.716,80.0,None,"['feature:Sex', 'fairness_metric_type:fairness', 'feature_value:female']",fairness,da94306a-8542-4b49-b04d-f27b7ae77ec1,aff300d0-f16a-45cc-8201-a00790c1df94,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937
2024-08-07 09:59:24.165078+00:00,fairness_value,c9333fb3-82cd-431d-9eb5-65bd64c9f7bd,100.0,80.0,None,"['feature:Age', 'fairness_metric_type:fairness', 'feature_value:18-25']",fairness,da94306a-8542-4b49-b04d-f27b7ae77ec1,aff300d0-f16a-45cc-8201-a00790c1df94,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


## Run drift monitor


Kick off a drift monitor run on current data. The monitor runs three hours, but can be manually initiated using the Python client, the REST API.

In [66]:
drift_run_details = wos_client.monitor_instances.run(monitor_instance_id=drift_monitor_instance_id, background_mode=False)




 Waiting for end of monitoring run bdbf70f7-e301-497c-a135-c49a4dec9d0b 




finished

---------------------------
 Successfully finished run 
---------------------------




In [67]:
time.sleep(5)

wos_client.monitor_instances.show_metrics(monitor_instance_id=drift_monitor_instance_id)

2024-08-07 09:59:46.400114+00:00,data_drift_magnitude,2080115b-d1d8-4bc5-aa20-56251ca634e5,0.0673076923076923,None,0.1,[],drift,505f727e-fa36-473e-91a5-e7f21cbabfc7,bdbf70f7-e301-497c-a135-c49a4dec9d0b,subscription,ad19393e-2078-4316-ac19-7de0ddfd7937


## Configure Explainability

Finally, we provide OpenScale with the training data to enable and configure the explainability features.

In [68]:
target = Target(
    target_type=TargetTypes.SUBSCRIPTION,
    target_id=subscription_id
)
parameters = {
    "enabled": True
}
explainability_details = wos_client.monitor_instances.create(
    data_mart_id=data_mart_id,
    background_mode=False,
    monitor_definition_id=wos_client.monitor_definitions.MONITORS.EXPLAINABILITY.ID,
    target=target,
    parameters=parameters
).result

explainability_monitor_id = explainability_details.metadata.id




 Waiting for end of monitor instance creation 021be1fb-c3a0-49dc-a841-8edd11749f70 




preparing.
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




## Run explanation for sample record

In [69]:
pl_records_resp = wos_client.data_sets.get_list_of_records(data_set_id=payload_data_set_id, limit=1, offset=0).result
scoring_ids = [pl_records_resp["records"][0]["entity"]["values"]["scoring_id"]]
print("Running explanations on scoring IDs: {}".format(scoring_ids))
explanation_types = ["lime", "contrastive"]
result = wos_client.monitor_instances.explanation_tasks(scoring_ids=scoring_ids, explanation_types=explanation_types, subscription_id=subscription_id).result
print(result)


Running explanations on scoring IDs: ['ce94d7419c28c598543294fbc4f5f864-1']
{
  "metadata": {
    "explanation_task_ids": [
      "7e2e2462-e51c-443f-96f1-d3ac3c87e6b5"
    ],
    "created_by": "1000331001",
    "created_at": "2024-08-07T10:00:21.852913Z"
  }
}


# Custom monitors and metrics <a name="custom"></a>

## Register custom monitor

In [70]:
def get_definition(monitor_name):
    monitor_definitions = wos_client.monitor_definitions.list().result.monitor_definitions
    
    for definition in monitor_definitions:
        if monitor_name == definition.entity.name:
            return definition
    
    return None

In [71]:
monitor_name = 'my model performance'
metrics = [MonitorMetricRequest(name='sensitivity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.8)]),
          MonitorMetricRequest(name='specificity',
                                thresholds=[MetricThreshold(type=MetricThresholdTypes.LOWER_LIMIT, default=0.75)])]
tags = [MonitorTagRequest(name='region', description='customer geographical region')]

existing_definition = get_definition(monitor_name)

if existing_definition is None:
    custom_monitor_details = wos_client.monitor_definitions.add(name=monitor_name, metrics=metrics, tags=tags, background_mode=False).result
else:
    custom_monitor_details = existing_definition

### Show available monitors types

In [72]:
wos_client.monitor_definitions.show()

my_model_performance,my model performance,"['sensitivity', 'specificity']"
sample_model_performance,Sample Model Performance,"['sensitivity', 'specificity', 'gender_less40_fav_prediction_ratio']"
custom_monitor_def_t5,custom_monitor_def_t5,"['specificityt5m1', 'sensitivityt5m2', 'specificityt5m3', 'sensitivityt5m4', 'specificityt5m5', 'sensitivityt5m6', 'specificityt5m7', 'sensitivityt5m8', 'specificityt5m9', 'sensitivityt5m0']"
custom_monitor_def_t4,custom_monitor_def_t4,['specificityt4m1']
custom_monitor_def_t3,custom_monitor_def_t3,"['sensitivityt3m2', 'specificityt3m1']"
custom_monitor_def_t2,custom_monitor_def_t2,"['sensitivityt2m2', 'specificityt2m1']"
custom_monitor_def_t1,custom_monitor_def_t1,"['sensitivityt1m2', 'specificityt1m1']"
fairness,Fairness,"['Fairness value', 'Average odds difference', 'False discovery rate difference', 'Error rate difference', 'False negative rate difference', 'False omission rate difference', 'False positive rate difference', 'True positive rate difference', 'Average absolute odds difference', 'Statistical parity difference', 'Impact score']"
model_health,Model health,"['Total scoring requests', 'Total records', 'Average records', 'Median records', 'Maximum records', 'Minimum records', 'Total payload size', 'Average payload size', 'Median payload size', 'Minimum payload size', 'Maximum payload size', 'Average API throughput', 'Minimum API throughput', 'Maximum API throughput', 'Median API throughput', 'Average API latency', 'Minimum API latency', 'Maximum API latency', 'Median API latency', 'Average record throughput', 'Minimum record throughput', 'Maximum record throughput', 'Median record throughput', 'Average record latency', 'Minimum record latency', 'Maximum record latency', 'Median record latency', 'Users', 'Errors', 'Data errors', 'System errors', 'Total input token count', 'Average input token count', 'Median input token count', 'Maximum input token count', 'Minimum input token count', 'Total output token count', 'Average output token count', 'Median output token count', 'Maximum output token count', 'Minimum output token count']"
performance,Performance,['Number of records']


Note: First 10 records were displayed.


### Get monitors uids and details

In [73]:
custom_monitor_id = custom_monitor_details.metadata.id

print(custom_monitor_id)

my_model_performance


In [74]:
custom_monitor_details = wos_client.monitor_definitions.get(monitor_definition_id=custom_monitor_id).result
print('Monitor definition details:', custom_monitor_details)

Monitor definition details: {
  "metadata": {
    "id": "my_model_performance",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_definition:my_model_performance",
    "url": "/v2/monitor_definitions/my_model_performance",
    "created_at": "2024-08-06T13:10:12.089000Z",
    "created_by": "cpadmin"
  },
  "entity": {
    "name": "my model performance",
    "metrics": [
      {
        "name": "sensitivity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.8
          }
        ],
        "expected_direction": "increasing",
        "id": "sensitivity"
      },
      {
        "name": "specificity",
        "thresholds": [
          {
            "type": "lower_limit",
            "default": 0.75
          }
        ],
        "expected_direction": "increasing",
        "id": "specificity"
      }
    ],
    "tags": [
      {
        "name": "region",
        "description": "customer geo

## Enable custom monitor for subscription

In [75]:
target = Target(
        target_type=TargetTypes.SUBSCRIPTION,
        target_id=subscription_id
    )

thresholds = [MetricThresholdOverride(metric_id='sensitivity', type = MetricThresholdTypes.LOWER_LIMIT, value=0.9)]

custom_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=custom_monitor_id,
            target=target, parameters={}
).result




 Waiting for end of monitor instance creation a342bc4a-3e77-4cd4-bb77-1f8c53d8bc32 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




### Get monitor instance id and configuration details

In [76]:
custom_monitor_instance_id = custom_monitor_instance_details.metadata.id

In [77]:
custom_monitor_instance_details = wos_client.monitor_instances.get(custom_monitor_instance_id).result
print(custom_monitor_instance_details)

{
  "metadata": {
    "id": "a342bc4a-3e77-4cd4-bb77-1f8c53d8bc32",
    "crn": "crn:v1:bluemix:public:aiopenscale:us-south:a/na:00000000-0000-0000-0000-000000000000:monitor_instance:a342bc4a-3e77-4cd4-bb77-1f8c53d8bc32",
    "url": "/v2/monitor_instances/a342bc4a-3e77-4cd4-bb77-1f8c53d8bc32",
    "created_at": "2024-08-07T10:00:43.046000Z",
    "created_by": "cpadmin",
    "modified_at": "2024-08-07T10:00:43.185000Z",
    "modified_by": "cpadmin"
  },
  "entity": {
    "data_mart_id": "00000000-0000-0000-0000-000000000000",
    "monitor_definition_id": "my_model_performance",
    "target": {
      "target_type": "subscription",
      "target_id": "ad19393e-2078-4316-ac19-7de0ddfd7937"
    },
    "parameters": {},
    "thresholds": [
      {
        "metric_id": "sensitivity",
        "type": "lower_limit",
        "value": 0.8
      },
      {
        "metric_id": "specificity",
        "type": "lower_limit",
        "value": 0.75
      }
    ],
    "status": {
      "state": "active"


## Storing custom metrics

In [78]:
from datetime import datetime, timezone, timedelta
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import MonitorMeasurementRequest
custom_monitoring_run_id = "ad19393e-2078-4316-ac19-7de0ddfd7937"
measurement_request = [MonitorMeasurementRequest(timestamp=datetime.now(timezone.utc), 
                                                 metrics=[{"specificity": 0.78, "sensitivity": 0.67, "region": "us-south"}], run_id=custom_monitoring_run_id)]
print(measurement_request[0])

{
  "timestamp": "2024-08-07T10:02:33.155972Z",
  "run_id": "ad19393e-2078-4316-ac19-7de0ddfd7937",
  "metrics": [
    {
      "specificity": 0.78,
      "sensitivity": 0.67,
      "region": "us-south"
    }
  ]
}


In [79]:
published_measurement_response = wos_client.monitor_instances.measurements.add(
    monitor_instance_id=custom_monitor_instance_id,
    monitor_measurement_request=measurement_request).result
published_measurement_id = published_measurement_response[0]["measurement_id"]
print(published_measurement_response)

[{'measurement_id': 'b7a8dc85-eb2f-419c-bca4-c66365a8bc5e', 'metrics': [{'region': 'us-south', 'sensitivity': 0.67, 'specificity': 0.78}], 'run_id': 'ad19393e-2078-4316-ac19-7de0ddfd7937', 'timestamp': '2024-08-07T10:02:33.155972Z'}]


### List and get custom metrics

In [80]:
time.sleep(5)
published_measurement = wos_client.monitor_instances.measurements.get(monitor_instance_id=custom_monitor_instance_id, measurement_id=published_measurement_id).result
print(published_measurement)

{
  "metadata": {
    "id": "b7a8dc85-eb2f-419c-bca4-c66365a8bc5e",
    "crn": "",
    "url": "/v2/monitor_instances/a342bc4a-3e77-4cd4-bb77-1f8c53d8bc32/measurements/b7a8dc85-eb2f-419c-bca4-c66365a8bc5e",
    "created_at": "2024-08-07T10:02:36.287930Z",
    "created_by": "N/A"
  },
  "entity": {
    "timestamp": "2024-08-07T10:02:33.155972Z",
    "run_id": "ad19393e-2078-4316-ac19-7de0ddfd7937",
    "values": [
      {
        "metrics": [
          {
            "id": "sensitivity",
            "value": 0.67,
            "lower_limit": 0.8
          },
          {
            "id": "specificity",
            "value": 0.78,
            "lower_limit": 0.75
          }
        ],
        "tags": [
          {
            "id": "region",
            "value": "us-south"
          }
        ]
      }
    ],
    "issue_count": 1,
    "target": {
      "target_type": "subscription",
      "target_id": "ad19393e-2078-4316-ac19-7de0ddfd7937"
    },
    "monitor_instance_id": "a342bc4a-3e77-4cd

## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [81]:
wos_client.data_sets.show_records(payload_data_set_id, limit=5)

,250,ce94d7419c28c598543294fbc4f5f864-1,1,less_100,2024-08-07T09:59:07.624427Z,0.971782546473715,less_0,car_new,No Risk,skilled,stores,none,4,19,all_credits_paid_back,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,savings_insurance,unemployed,"[0.971782546473715, 0.028217453526285]"
,250,ce94d7419c28c598543294fbc4f5f864-10,2,100_to_500,2024-08-07T09:59:07.624427Z,0.9699113373984984,0_to_200,car_new,No Risk,skilled,stores,none,15,19,credits_paid_to_date,2,yes,rent,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,male,1,real_estate,1_to_4,"[0.9699113373984984, 0.030088662601501545]"
,5461,ce94d7419c28c598543294fbc4f5f864-100,4,unknown,2024-08-07T09:59:07.624427Z,0.6003104464290147,no_checking,appliances,No Risk,skilled,none,none,36,43,prior_payments_delayed,4,yes,free,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,yes,male,2,car_other,4_to_7,"[0.6003104464290147, 0.39968955357098535]"
,2885,ce94d7419c28c598543294fbc4f5f864-101,3,less_100,2024-08-07T09:59:07.624427Z,0.8271092517948442,0_to_200,car_used,No Risk,skilled,none,none,11,42,prior_payments_delayed,2,yes,own,2,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,car_other,less_1,"[0.8271092517948442, 0.17289074820515588]"
,250,ce94d7419c28c598543294fbc4f5f864-102,2,less_100,2024-08-07T09:59:07.624427Z,0.98649525682623,less_0,car_new,No Risk,skilled,none,none,6,19,all_credits_paid_back,2,yes,own,1,edc1e06c-7bf9-4522-8893-05a919a35ca0,none,female,1,real_estate,less_1,"[0.98649525682623, 0.01350474317377005]"


## Insert historical fairness metrics

In [82]:
historyDays = 7

In [83]:
!rm history_fairness_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_fairness_v2.json
!ls -lh history_fairness_v2.json

rm: history_fairness_v2.json: No such file or directory
-rw-r--r--  1 nelwin  staff   1.1M Aug  7 15:33 history_fairness_v2.json


In [84]:
from datetime import datetime, timedelta, timezone

from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Measurements
with open("history_fairness_v2.json") as f:
    fairness_values = json.load(f)
    for day in range(historyDays):
        print('Loading day', day + 1)
        daily_measurement_requests = []
        sources_list = []
        for hour in range(24):
            score_time = (datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
            index = (day * 24 + hour) % len(fairness_values) # wrap around and reuse values if needed
            fairness_values[index]["timestamp"] = score_time
            #print(score_time) 
            fairness_value = fairness_values[index]
            metrics_list = fairness_value["metrics"]
            sources = fairness_value["sources"]
            sources_list = []
            for source in sources:
                source_id = source["id"]
                source_type = source["type"]
                source_data = source["data"]
                if source_id == "bias_detection_summary":
                    source_data["evaluated_at"] = score_time
                    source_data["favourable_class"] = ["No Risk"]
                    source_data["unfavourable_class"] = ["Risk"]
                    source_data["score_type"] = "disparate impact"
                sources_list.append(
                    Source(
                        id=source_id,
                        type=source_type,
                        data=source_data
                    )
                )  
            measurement_request = MonitorMeasurementRequest(metrics=metrics_list, sources=sources_list, timestamp=score_time)
            daily_measurement_requests.append(measurement_request)
        measurements_client = Measurements(wos_client)
        measurements_client.add(monitor_instance_id=fairness_monitor_instance_id, monitor_measurement_request=daily_measurement_requests)     
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical debias metrics

In [85]:
!rm history_debias_v2.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_debias_v2.json
!ls -lh history_debias_v2.json

rm: history_debias_v2.json: No such file or directory
-rw-r--r--  1 nelwin  staff   725K Aug  7 15:33 history_debias_v2.json


In [86]:
with open("history_debias_v2.json") as f:
    debias_values = json.load(f)
    for day in range(historyDays):
        print('Loading day', day + 1)
        daily_measurement_requests = []
        sources_list = []
        for hour in range(24):
            score_time = (datetime.now(timezone.utc) + timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
            index = (day * 24 + hour) % len(debias_values) # wrap around and reuse values if needed
            debias_values[index]["timestamp"] = score_time
            debias_value = debias_values[index]
            metrics_list = debias_value["metrics"]
            sources = debias_value["sources"]
            sources_list = []
            for source in sources:
                sources_list.append(
                    Source(
                        id=source["id"],
                        type=source["type"],
                        data=source["data"]
                    )
                )  
            measurement_request = MonitorMeasurementRequest(metrics=metrics_list, sources=sources_list, timestamp=score_time)
            daily_measurement_requests.append(measurement_request)
        measurements_client = Measurements(wos_client)
        measurements_client.add(monitor_instance_id=fairness_monitor_instance_id, monitor_measurement_request=daily_measurement_requests)         

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical quality metrics

In [87]:
measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    quality_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        
        metric = {"area_under_roc": measurements[day]}
                
        measurement_request = MonitorMeasurementRequest(timestamp=score_time,metrics = [metric])
        quality_measurement_requests.append(measurement_request)
        
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=quality_monitor_instance_id,
                                            monitor_measurement_request=quality_measurement_requests).result    
    
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical confusion matrixes

In [88]:
!rm history_quality_metrics.json
with io.capture_output() as captured:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_quality_metrics.json
!ls -lh history_quality_metrics.json

rm: history_quality_metrics.json: No such file or directory
-rw-r--r--  1 nelwin  staff    78K Aug  7 15:33 history_quality_metrics.json


In [89]:
from ibm_watson_openscale.base_classes.watson_open_scale_v2 import Source

with open('history_quality_metrics.json') as json_file:
    records = json.load(json_file)
    
for day in range(historyDays):
    index = 0
    cm_measurement_requests = []
    print('Loading day', day + 1)
    
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"

        metric = records[index]['metrics']
        source = records[index]['sources']

        
        measurement_request = {"timestamp": score_time, "metrics": [metric], "sources": [source]}
        cm_measurement_requests.append(measurement_request)

        index+=1

    response = wos_client.monitor_instances.measurements.add(monitor_instance_id=quality_monitor_instance_id, monitor_measurement_request=cm_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical performance metrics

In [90]:
target = Target(
        target_type=TargetTypes.INSTANCE,
        target_id=payload_data_set_id
    )


performance_monitor_instance_details = wos_client.monitor_instances.create(
            data_mart_id=data_mart_id,
            background_mode=False,
            monitor_definition_id=wos_client.monitor_definitions.MONITORS.PERFORMANCE.ID,
            target=target, parameters={}
).result
performance_monitor_instance_id = performance_monitor_instance_details.metadata.id




 Waiting for end of monitor instance creation eb5ba388-f6b5-49fd-85ae-8bb23fe8bdbd 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------




In [91]:
for day in range(historyDays):
    performance_measurement_requests = []
    print('Loading day', day + 1)
    for hour in range(24):
        score_time = datetime.utcnow() + timedelta(hours=(-(24*day + hour + 1)))
        score_time = score_time.isoformat() + "Z"
        score_count = random.randint(60, 600)
        
        metric = {"record_count": score_count, "data_set_type": "scoring_payload"}
        
        measurement_request = {"timestamp": score_time, "metrics": [metric]}
        performance_measurement_requests.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=performance_monitor_instance_id,
                                            monitor_measurement_request=performance_measurement_requests).result    

print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


## Insert historical drift measurements

In [92]:
!rm history_drift_measurement_*.json
with io.capture_output() as captured_0:
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_0.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_1.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_2.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_3.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_4.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_5.json
    !wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/historical_data/credit_risk/history_drift_measurement_6.json
!ls -lh history_drift_measurement_*.json

zsh:1: no matches found: history_drift_measurement_*.json
-rw-r--r--  1 nelwin  staff   831K Aug  7 15:33 history_drift_measurement_0.json
-rw-r--r--  1 nelwin  staff   867K Aug  7 15:33 history_drift_measurement_1.json
-rw-r--r--  1 nelwin  staff   870K Aug  7 15:33 history_drift_measurement_2.json
-rw-r--r--  1 nelwin  staff   909K Aug  7 15:33 history_drift_measurement_3.json
-rw-r--r--  1 nelwin  staff   840K Aug  7 15:33 history_drift_measurement_4.json
-rw-r--r--  1 nelwin  staff   835K Aug  7 15:33 history_drift_measurement_5.json
-rw-r--r--  1 nelwin  staff   840K Aug  7 15:33 history_drift_measurement_6.json


In [93]:
for day in range(historyDays):
    drift_measurements = []

    with open("history_drift_measurement_{}.json".format(day), 'r') as history_file:
        drift_daily_measurements = json.load(history_file)
    print('Loading day', day + 1)

    #Historical data contains 8 records per day - each represents 3 hour drift window.
    
    for nb_window, records in enumerate(drift_daily_measurements):
        for record in records:
            window_start =  datetime.utcnow() + timedelta(hours=(-(24 * day + (nb_window+1)*3 + 1))) # first_payload_record_timestamp_in_window (oldest)
            window_end = datetime.utcnow() + timedelta(hours=(-(24 * day + nb_window*3 + 1)))# last_payload_record_timestamp_in_window (most recent)
            #modify start and end time for each record
            record['sources'][0]['data']['start'] = window_start.isoformat() + "Z"
            record['sources'][0]['data']['end'] = window_end.isoformat() + "Z"
            
            
            metric = record['metrics'][0]
            source = record['sources'][0]

            measurement_request = {"timestamp": window_start.isoformat() + "Z", "metrics": [metric], "sources": [source]}
            
            drift_measurements.append(measurement_request)
        
    response = wos_client.monitor_instances.measurements.add(
                                            monitor_instance_id=drift_monitor_instance_id,
                                            monitor_measurement_request=drift_measurements).result    

    
    print("Daily loading finished.")

Loading day 1
Daily loading finished.
Loading day 2
Daily loading finished.
Loading day 3
Daily loading finished.
Loading day 4
Daily loading finished.
Loading day 5
Daily loading finished.
Loading day 6
Daily loading finished.
Loading day 7
Daily loading finished.


In [94]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)

Datamart: 00000000-0000-0000-0000-000000000000
Model: 1b9f22c5-ac03-4017-b40b-27969e429379
Deployment: edc1e06c-7bf9-4522-8893-05a919a35ca0


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and can be deleted if necessary.